In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import FwFM

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo

In [4]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [5]:
def gen_pattern_replace_and_matcher_for_FwFM(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,getitem_node_names,num_field,batch,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  print(shape_info)
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)
      def forward(self,x):
        x = self.embed(x)
        cross_term = self.pn(x)
        # x = x.view(-1,self.embed_output_dim)
        return self.mlp(x.view(-1,self.embed_output_dim)), cross_term
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  if linear_node_module.bias is None:
      linear_node_bias = None
  else:
      linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      if linear_node_bias is not None:
         self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
         self.redency_linear.bias.data.copy_(linear_node_bias)
      else:
         self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0],False)
      self.redency_linear.weight.data.copy_(redency_weight)
      

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total   
         
    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim= -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = -1)
    def forward(self,x):
      redency_part = x[redency_part_slice] 
      unredency_part = x[unredency_part_slice] 
      redency_embed = self.embed(redency_part)
      unredency_embed = self.embed(unredency_part)
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redency_embed,unredency_embed)
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)
      return self.redency_linear(redency_embed.view(-1,self.redency_weight_len)) + self.unredency_linear(unredency_embed.view(-1,self.unredency_weight_len)), cross_term
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [6]:
def workload_FwFM(num_field, batch,prefix,dim = 64):
  print(f"now gen workload of FwFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  FwFM_model_ori = FwFM.FwFM([100 for i in range(num_field)],dim)
  ori_traced = symbolic_trace(FwFM_model_ori)
  
  FwFM_model_modify = FwFM.FwFM([100 for i in range(num_field)],dim)
  modify_traced = symbolic_trace(FwFM_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FwFM(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      key_node_name = "linear_weight_layer",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field, batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FwFM,l = [1024,512,256]):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(100):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of FwFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,batch,prefix,dim)
  run(ori)
  run(modify)


In [8]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50
torch.Size([1, 3520])
(8.351431846618652, 0.0009913585168500763)
(5.638528001551726, 0.0016345559158280258)


In [9]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50
torch.Size([1, 3520])
(16.685952673153, 0.013334046504705365)
(11.162645262114856, 0.011292010738704457)


In [10]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50
torch.Size([1, 3520])
(33.63669379876585, 0.006665304454974937)
(22.27910754145408, 0.017223168762497486)


In [11]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of FwFM with config: dim: 32, num_field: 110, prefix: 50
torch.Size([1, 3520])
(67.37620240814832, 0.03598563513231478)
(44.64777086219009, 0.058032349699399355)


In [12]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50
torch.Size([1, 7040])
(14.911987294956129, 0.0058915464886439015)
(9.648988772411736, 0.013502247226163402)


In [13]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50
torch.Size([1, 7040])
(29.948061008842625, 0.009702147639955179)
(19.236011524589696, 0.027614302256885766)


In [14]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50
torch.Size([1, 7040])
(59.977481024605886, 0.00706526604239005)
(38.477898889658405, 0.047898292665573966)


In [15]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of FwFM with config: dim: 64, num_field: 110, prefix: 50
torch.Size([1, 7040])
(121.08099762274294, 0.03836301136765419)
(77.74960249297473, 0.16706528281376012)


In [16]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145
torch.Size([1, 5440])
(20.581085750034877, 0.041910401981502754)
(4.549808346495336, 0.0011765435189428173)


In [17]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145
torch.Size([1, 5440])
(41.368238293394754, 0.05639701046034401)
(8.95925979225003, 0.00949980577611145)


In [18]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145
torch.Size([1, 5440])
(82.47860507575832, 0.03071788538040723)
(17.822402954101562, 0.010711759446148418)


In [19]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of FwFM with config: dim: 32, num_field: 170, prefix: 145
torch.Size([1, 5440])
(165.70566247434033, 0.08317313939717455)
(35.719618505361126, 0.02690595083261053)


In [20]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145
torch.Size([1, 10880])
(36.59390760927784, 0.05126174944957714)
(7.500254378026845, 0.0028480096640569165)


In [21]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145
torch.Size([1, 10880])
(72.96179129152881, 0.053677529021867995)
(14.922867911202568, 0.02582180898626779)


In [22]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145
torch.Size([1, 10880])
(147.46600450788225, 0.11418876259668483)
(29.8620385539775, 0.012872070894794902)


In [23]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145, batch :8192
now gen workload of FwFM with config: dim: 64, num_field: 170, prefix: 145
torch.Size([1, 10880])


Traceback (most recent call last):
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph_module.py", line 304, in __call__
    return super(self.cls, obj).__call__(*args, **kwargs)  # type: ignore[misc]
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1522, in _slow_forward
    result = self.forward(*input, **kwargs)
  File "<eval_with_key>.90 from /home/yssun/pytorch-fm/torchfm/model/test_fx/exp/../../FwFM.py:31 in forward", line 9, in forward
    getitem_1 = embedding_embedding[(slice(None, None, None), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.06 GiB. GPU 